## **NAME**    : Gajula Sai Chaitanya
## **ROLL NO** : 18CS30018

In [1]:
!pip install git+https://github.com/data61/python-paillier.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/data61/python-paillier.git to /tmp/pip-req-build-71bi6qc0
  Running command git clone --filter=blob:none --quiet https://github.com/data61/python-paillier.git /tmp/pip-req-build-71bi6qc0
  Resolved https://github.com/data61/python-paillier.git to commit 7d9911eb03c3c2d64399bc15405feb5e628379d1
  Preparing metadata (setup.py) ... done


### Part - 1: Running the codes and recording timing for Federated Learning

In [2]:
#!/usr/bin/env python3.4
import math

import phe.encoding
from phe import paillier


class ExampleEncodedNumber(phe.encoding.EncodedNumber):
    BASE = 64
    LOG2_BASE = math.log(BASE, 2)


print("Generating paillier keypair")
public_key, private_key = paillier.generate_paillier_keypair()


def encode_and_encrypt_example():
    print("Encoding a large positive number. With a BASE {} encoding scheme".format(ExampleEncodedNumber.BASE))
    encoded = ExampleEncodedNumber.encode(public_key, 2.1 ** 20)
    print("Checking that decoding gives the same number...")
    assert 2.1 ** 20 == encoded.decode()

    print("Encrypting the encoded number")
    encrypted = public_key.encrypt(encoded)

    print("Decrypting...")
    decrypted_but_encoded = \
        private_key.decrypt_encoded(encrypted, ExampleEncodedNumber)

    print("Checking the decrypted number is what we started with")
    assert abs(2.1 ** 20 - decrypted_but_encoded.decode()) < 1e-12


def math_example():
    print("Encoding two large positive numbers. BASE={}".format(ExampleEncodedNumber.BASE))

    a = 102545 + (64 ** 8)
    b = 123 + (8 ** 20)

    encoded_a = ExampleEncodedNumber.encode(public_key, a)
    encoded_b = ExampleEncodedNumber.encode(public_key, b)

    print("Checking that decoding gives the same number...")
    assert a == encoded_a.decode()
    assert b == encoded_b.decode()

    print("Encrypting the encoded numbers")
    encrypted_a = public_key.encrypt(encoded_a)
    encrypted_b = public_key.encrypt(encoded_b)

    print("Adding the encrypted numbers")
    encrypted_c = encrypted_a + encrypted_b

    print("Decrypting the one encrypted sum")
    decrypted_but_encoded = \
        private_key.decrypt_encoded(encrypted_c, ExampleEncodedNumber)

    print("Checking the decrypted number is what we started with")

    print("Decrypted: {}".format(decrypted_but_encoded.decode()))
    assert abs((a + b) - decrypted_but_encoded.decode()) < 1e-15


if __name__ == "__main__":
    encode_and_encrypt_example()

    math_example()

Generating paillier keypair
Encoding a large positive number. With a BASE 64 encoding scheme
Checking that decoding gives the same number...
Encrypting the encoded number
Decrypting...
Checking the decrypted number is what we started with
Encoding two large positive numbers. BASE=64
Checking that decoding gives the same number...
Encrypting the encoded numbers
Adding the encrypted numbers
Decrypting the one encrypted sum
Checking the decrypted number is what we started with
Decrypted: 1153202979583660300


In [3]:
"""
This example involves learning using sensitive medical data from multiple hospitals
to predict diabetes progression in patients. The data is a standard dataset from
sklearn[1].

Recorded variables are:
- age,
- gender,
- body mass index,
- average blood pressure,
- and six blood serum measurements.

The target variable is a quantitative measure of the disease progression.
Since this measure is continuous, we solve the problem using linear regression.

The patients' data is split between 3 hospitals, all sharing the same features
but different entities. We refer to this scenario as horizontally partitioned.

The objective is to make use of the whole (virtual) training set to improve
upon the model that can be trained locally at each hospital.

50 patients will be kept as a test set and not used for training.

An additional agent is the 'server' who facilitates the information exchange
among the hospitals under the following privacy constraints:

1) The individual patient's record at each hospital cannot leave the premises,
   not even in encrypted form.
2) Information derived (read: gradients) from any hospital's dataset
   cannot be shared, unless it is first encrypted.
3) None of the parties (hospitals AND server) should be able to infer WHERE
   (in which hospital) a patient in the training set has been treated.

Note that we do not protect from inferring IF a particular patient's data
has been used during learning. Differential privacy could be used on top of
our protocol for addressing the problem. For simplicity, we do not discuss
it in this example.

In this example linear regression is solved by gradient descent. The server
creates a paillier public/private keypair and does not share the private key.
The hospital clients are given the public key. The protocol works as follows.
Until convergence: hospital 1 computes its gradient, encrypts it and sends it
to hospital 2; hospital 2 computes its gradient, encrypts and sums it to
hospital 1's; hospital 3 does the same and passes the overall sum to the
server. The server obtains the gradient of the whole (virtual) training set;
decrypts it and sends the gradient back - in the clear - to every client.
The clients then update their respective local models.

From the learning viewpoint, notice that we are NOT assuming that each
hospital sees an unbiased sample from the same patients' distribution:
hospitals could be geographically very distant or serve a diverse population.
We simulate this condition by sampling patients NOT uniformly at random,
but in a biased fashion.
The test set is instead an unbiased sample from the overall distribution.

From the security viewpoint, we consider all parties to be "honest but curious".
Even by seeing the aggregated gradient in the clear, no participant can pinpoint
where patients' data originated. This is true if this RING protocol is run by
at least 3 clients, which prevents reconstruction of each others' gradients
by simple difference.

This example was inspired by Google's work on secure protocols for federated
learning[2].

[1]: http://scikit-learn.org/stable/datasets/index.html#diabetes-dataset
[2]: https://research.googleblog.com/2017/04/federated-learning-collaborative.html

Dependencies: numpy, sklearn
"""

import numpy as np
from sklearn.datasets import load_diabetes

import phe as paillier

seed = 43
np.random.seed(seed)
import time


def get_data(n_clients):
    """
    Import the dataset via sklearn, shuffle and split train/test.
    Return training, target lists for `n_clients` and a holdout test set
    """
    print("Loading data")
    diabetes = load_diabetes()
    y = diabetes.target
    X = diabetes.data
    # Add constant to emulate intercept
    X = np.c_[X, np.ones(X.shape[0])]

    # The features are already preprocessed
    # Shuffle
    perm = np.random.permutation(X.shape[0])
    X, y = X[perm, :], y[perm]

    # Select test at random
    test_size = 50
    test_idx = np.random.choice(X.shape[0], size=test_size, replace=False)
    train_idx = np.ones(X.shape[0], dtype=bool)
    train_idx[test_idx] = False
    X_test, y_test = X[test_idx, :], y[test_idx]
    X_train, y_train = X[train_idx, :], y[train_idx]

    # Split train among multiple clients.
    # The selection is not at random. We simulate the fact that each client
    # sees a potentially very different sample of patients.
    X, y = [], []
    step = int(X_train.shape[0] / n_clients)
    for c in range(n_clients):
        X.append(X_train[step * c: step * (c + 1), :])
        y.append(y_train[step * c: step * (c + 1)])

    return X, y, X_test, y_test


def mean_square_error(y_pred, y):
    """ 1/m * \sum_{i=1..m} (y_pred_i - y_i)^2 """
    return np.mean((y - y_pred) ** 2)


def encrypt_vector(public_key, x):
    return [public_key.encrypt(i) for i in x]


def decrypt_vector(private_key, x):
    return np.array([private_key.decrypt(i) for i in x])


def sum_encrypted_vectors(x, y):
    if len(x) != len(y):
        raise ValueError('Encrypted vectors must have the same size')
    return [x[i] + y[i] for i in range(len(x))]


class Server:
    """Private key holder. Decrypts the average gradient"""

    def __init__(self, key_length):
         keypair = paillier.generate_paillier_keypair(n_length=key_length)
         self.pubkey, self.privkey = keypair

    def decrypt_aggregate(self, input_model, n_clients):
        return decrypt_vector(self.privkey, input_model) / n_clients


class Client:
    """Runs linear regression with local data or by gradient steps,
    where gradient can be passed in.

    Using public key can encrypt locally computed gradients.
    """

    def __init__(self, name, X, y, pubkey):
        self.name = name
        self.pubkey = pubkey
        self.X, self.y = X, y
        self.weights = np.zeros(X.shape[1])

    def fit(self, n_iter, eta=0.01):
        """Linear regression for n_iter"""
        for _ in range(n_iter):
            gradient = self.compute_gradient()
            self.gradient_step(gradient, eta)

    def gradient_step(self, gradient, eta=0.01):
        """Update the model with the given gradient"""
        self.weights -= eta * gradient

    def compute_gradient(self):
        """Compute the gradient of the current model using the training set
        """
        delta = self.predict(self.X) - self.y
        return delta.dot(self.X) / len(self.X)

    def predict(self, X):
        """Score test data"""
        return X.dot(self.weights)

    def encrypted_gradient(self, sum_to=None):
        """Compute and encrypt gradient.

        When `sum_to` is given, sum the encrypted gradient to it, assumed
        to be another vector of the same size
        """
        gradient = self.compute_gradient()
        encrypted_gradient = encrypt_vector(self.pubkey, gradient)

        if sum_to is not None:
            return sum_encrypted_vectors(sum_to, encrypted_gradient)
        else:
            return encrypted_gradient


def federated_learning(X, y, X_test, y_test, config):
    n_clients = config['n_clients']
    n_iter = config['n_iter']
    names = ['Hospital {}'.format(i) for i in range(1, n_clients + 1)]

    # Instantiate the server and generate private and public keys
    # NOTE: using smaller keys sizes wouldn't be cryptographically safe
    server = Server(key_length=config['key_length'])

    # Instantiate the clients.
    # Each client gets the public key at creation and its own local dataset
    clients = []
    for i in range(n_clients):
        clients.append(Client(names[i], X[i], y[i], server.pubkey))

    # The federated learning with gradient descent
    print('Running distributed gradient aggregation for {:d} iterations'
          .format(n_iter))
    for i in range(n_iter):

        # Compute gradients, encrypt and aggregate
        encrypt_aggr = clients[0].encrypted_gradient(sum_to=None)
        for c in clients[1:]:
            encrypt_aggr = c.encrypted_gradient(sum_to=encrypt_aggr)

        # Send aggregate to server and decrypt it
        aggr = server.decrypt_aggregate(encrypt_aggr, n_clients)

        # Take gradient steps
        for c in clients:
            c.gradient_step(aggr, config['eta'])

    print('Error (MSE) that each client gets after running the protocol:')
    start = time.time()
    for c in clients:
        y_pred = c.predict(X_test)
        mse = mean_square_error(y_pred, y_test)
        print('{:s}:\t{:.2f}'.format(c.name, mse))
    end = time.time()
    print("Time required for Single Prediction (Local learning): ", (end-start)/(X_test.shape[0]))


def local_learning(X, y, X_test, y_test, config):
    n_clients = config['n_clients']
    names = ['Hospital {}'.format(i) for i in range(1, n_clients + 1)]

    # Instantiate the clients.
    # Each client gets the public key at creation and its own local dataset
    clients = []
    for i in range(n_clients):
        clients.append(Client(names[i], X[i], y[i], None))

    # Each client trains a linear regressor on its own data
    print('Error (MSE) that each client gets on test set by '
          'training only on own local data:')
    start = time.time()
    for c in clients:
        c.fit(config['n_iter'], config['eta'])
        y_pred = c.predict(X_test)
        mse = mean_square_error(y_pred, y_test)
        print('{:s}:\t{:.2f}'.format(c.name, mse))
    end = time.time()
    print("Time required for Single Prediction (Local learning): ", (end-start)/(X_test.shape[0]))


if __name__ == '__main__':
    config = {
        'n_clients': 5,
        'key_length': 1024,
        'n_iter': 50,
        'eta': 1.5,
    }
    # load data, train/test split and split training data between clients
    X, y, X_test, y_test = get_data(n_clients=config['n_clients'])
    # first each hospital learns a model on its respective dataset for comparison.
    print("################################ LOCAL LEARNING (Linear Regression) #############################")
    local_learning(X, y, X_test, y_test, config)
    # and now the full glory of federated learning
    print("################################ FEDERATED LEARNING (Linear Regression) #############################")
    start = time.time()
    federated_learning(X, y, X_test, y_test, config)
    end = time.time()
    print("Time required for Federated Learning Process: ", (end-start)/(X_test.shape[0]))

Loading data
################################ LOCAL LEARNING (Linear Regression) #############################
Error (MSE) that each client gets on test set by training only on own local data:
Hospital 1:	3810.44
Hospital 2:	3982.58
Hospital 3:	3569.32
Hospital 4:	4144.15
Hospital 5:	3848.39
Time required for Single Prediction (Local learning):  0.00011647224426269531
################################ FEDERATED LEARNING (Linear Regression) #############################
Running distributed gradient aggregation for 50 iterations
Error (MSE) that each client gets after running the protocol:
Hospital 1:	3775.50
Hospital 2:	3775.50
Hospital 3:	3775.50
Hospital 4:	3775.50
Hospital 5:	3775.50
Time required for Single Prediction (Local learning):  6.678104400634766e-05
Time required for Federated Learning Process:  1.1341312265396117


### Part - 2: Partial Privacy Preserving SVM 

* Implementation of Privacy Preserving SVM on a spam classification dataset

In [4]:
from numpy.lib.function_base import gradient
import time
import os.path
from zipfile import ZipFile
from urllib.request import urlopen
from contextlib import contextmanager

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

import phe as paillier

from tqdm import tqdm

np.random.seed(42)

# Enron spam dataset hosted by https://cloudstor.aarnet.edu.au
url = [
    'https://cloudstor.aarnet.edu.au/plus/index.php/s/RpHZ57z2E3BTiSQ/download',
    'https://cloudstor.aarnet.edu.au/plus/index.php/s/QVD4Xk5Cz3UVYLp/download'
]


def download_data():
    """Download two sets of Enron1 spam/ham e-mails if they are not here
    We will use the first as trainset and the second as testset.
    Return the path prefix to us to load the data from disk."""

    n_datasets = 2
    for d in range(1, n_datasets + 1):
        if not os.path.isdir('enron%d' % d):

            URL = url[d-1]
            print("Downloading %d/%d: %s" % (d, n_datasets, URL))
            folderzip = 'enron%d.zip' % d

            with urlopen(URL) as remotedata:
                with open(folderzip, 'wb') as z:
                    z.write(remotedata.read())

            with ZipFile(folderzip) as z:
                z.extractall()
            os.remove(folderzip)


def preprocess_data(config):
    """
    Get the Enron e-mails from disk.
    Represent them as bag-of-words.
    Shuffle and split train/test.
    """
    print("Importing dataset from disk...")
    path = 'enron1/ham/'
    ham1 = [open(path + f, 'r', errors='replace').read().strip(r"\n")
            for f in os.listdir(path) if os.path.isfile(path + f)]
    path = 'enron1/spam/'
    spam1 = [open(path + f, 'r', errors='replace').read().strip(r"\n")
             for f in os.listdir(path) if os.path.isfile(path + f)]
    path = 'enron2/ham/'
    ham2 = [open(path + f, 'r', errors='replace').read().strip(r"\n")
            for f in os.listdir(path) if os.path.isfile(path + f)]
    path = 'enron2/spam/'
    spam2 = [open(path + f, 'r', errors='replace').read().strip(r"\n")
             for f in os.listdir(path) if os.path.isfile(path + f)]

    # Merge and create labels
    emails = ham1 + spam1 + ham2 + spam2
    y = np.array([-1] * len(ham1) + [1] * len(spam1) +
                 [-1] * len(ham2) + [1] * len(spam2))

    # Words count, keep only frequent words
    count_vect = CountVectorizer(decode_error='replace', stop_words='english',
                                 min_df=0.001)
    X = count_vect.fit_transform(emails)
    X = X.toarray()

    print('Vocabulary size: %d' % X.shape[1])

    # Shuffle
    perm = np.random.permutation(X.shape[0])
    X, y = X[perm, :], y[perm]

    # Split train and test
    split = 50
    X_train, X_test = X[-split:, :], X[:-split, :]
    y_train, y_test = y[-split:], y[:-split]
    print(X_train.shape, X_test.shape)

    print("Labels in trainset are {:.2f} spam : {:.2f} ham".format(
        np.mean(y_train == 1), np.mean(y_train == -1)))
    
    # Split train among multiple clients.
    # The selection is not at random. We simulate the fact that each client
    # sees a potentially very different sample of patients.
    X, y = [], []
    step = int(X_train.shape[0] / config['n_clients'])
    for c in range(config['n_clients']):
        X.append(X_train[step * c: step * (c + 1), :])
        y.append(y_train[step * c: step * (c + 1)])

    return X, y, X_test, y_test


@contextmanager
def timer():
    """Helper for measuring runtime"""

    time0 = time.perf_counter()
    yield
    print('[elapsed time: %.2f s]' % (time.perf_counter() - time0))

def mean_square_error(y_pred, y):
    """ 1/m * \sum_{i=1..m} (y_pred_i - y_i)^2 """
    return np.mean((y - y_pred) ** 2)


def encrypt_vector(public_key, x):
    try:
        return np.array([public_key.encrypt(i) for i in x])
    except:
        return public_key.encrypt(int(x))

def decrypt_vector(private_key, x):
    try:
        return np.array([private_key.decrypt(i) for i in x])
    except:
         return private_key.decrypt(x)

def sum_encrypted_vectors(x, y):
    # print(type(x), type(y))
    try:
        if len(x) != len(y):
            raise ValueError('Encrypted vectors must have the same size')
        return np.array([x[i] + y[i] for i in range(len(x))])
    except:
        return x + y


class Server:
    """Private key holder. Decrypts the average gradient"""

    def __init__(self, key_length):
         keypair = paillier.generate_paillier_keypair(n_length=key_length)
         self.pubkey, self.privkey = keypair

    def decrypt_aggregate(self, input_model, input_model_2, n_clients):
        if input_model_2:
            return decrypt_vector(self.privkey, input_model) / n_clients, decrypt_vector(self.privkey, input_model_2) / n_clients
        return decrypt_vector(self.privkey, input_model) / n_clients, None


class Client:
    """Runs linear regression with local data or by gradient steps,
    where gradient can be passed in.

    Using public key can encrypt locally computed gradients.
    """

    def __init__(self, name, X, y, pubkey, lr=0.01, lambda_param=0.01):
        self.name = name
        self.pubkey = pubkey
        self.X, self.y = X, y
        self.learning_rate = lr
        self.lambda_param = lambda_param
        n_samples, n_features = self.X.shape
        self.w = np.zeros(n_features)
        self.b = 0

    def fit(self, n_iter=50):
        # Perform gradient descent to optimize the SVM
        for i in tqdm(range(n_iter), desc='local_learn_iters'):
            for idx, x_i in enumerate(self.X):
                gradient_w, gradient_b = self.compute_gradient(idx, x_i)
                self.gradient_step(gradient_w, gradient_b)

    def gradient_step(self, gradient_w, gradient_b):
        """Update the model with the given gradient"""
        self.w -= self.learning_rate * gradient_w
        if gradient_b:
           self.b -= self.learning_rate * gradient_b

    def compute_gradient(self, idx, x_i):
        """Compute the gradient of the current model using the training set
        """
        # print(self.y[idx] * (np.dot(x_i, self.w) - self.b))
        condition = self.y[idx] * (np.dot(x_i, self.w) - self.b) >= 1
        # print(condition)
        if condition:
            gradient_w = 2 * self.lambda_param * self.w
            return gradient_w, None
        else:
            gradient_w = 2 * self.lambda_param * self.w - np.dot(x_i, self.y[idx])
            gradient_b = self.y[idx]
            return gradient_w, gradient_b

    def predict(self, X):
        # Make predictions on new data points
        linear_output = np.dot(X, self.w) - self.b
        return np.sign(linear_output)
    

    def encrypted_gradient(self, idx, x_i, sum_to_w=None, sum_to_b=None):
        """Compute and encrypt gradient.

        When `sum_to` is given, sum the encrypted gradient to it, assumed
        to be another vector of the same size
        """
        g_w, g_b = self.compute_gradient(idx, x_i)
        # print(type(g_w), type(g_b))
        # print('   INNER CHECKPOINT 0')
        encrypted_g_w = encrypt_vector(self.pubkey, g_w)
        # print("   INNER CHECKPOINT 1")
        # print(g_b)
        encrypted_g_b = None
        if g_b:
            encrypted_g_b = encrypt_vector(self.pubkey, g_b)
        # print("   INNER CHECKPOINT 2")

        if sum_to_w is not None and sum_to_b is not None:
            if g_b and encrypted_g_b: 
               return sum_encrypted_vectors(sum_to_w, encrypted_g_w), sum_encrypted_vectors(sum_to_b, encrypted_g_b)
            return sum_encrypted_vectors(sum_to_w, encrypted_g_w), None
        else:
            if g_b and encrypted_g_b: 
                return encrypted_g_w, encrypted_g_b
            return encrypted_g_w, None


def federated_learning(X, y, X_test, y_test, config):
    n_clients = config['n_clients']
    n_iter = config['n_iter']
    names = ['Hospital {}'.format(i) for i in range(1, n_clients + 1)]

    # Instantiate the server and generate private and public keys
    # NOTE: using smaller keys sizes wouldn't be cryptographically safe
    server = Server(key_length=config['key_length'])

    # Instantiate the clients.
    # Each client gets the public key at creation and its own local dataset
    clients = []
    for i in range(n_clients):
        clients.append(Client(names[i], X[i], y[i], server.pubkey))

    # The federated learning with gradient descent
    print('Running distributed gradient aggregation for {:d} iterations'.format(n_iter))

    for i in tqdm(range(n_iter), desc='fed_learn_iters'):
          # Compute gradients, encrypt and aggregate
          for idx in tqdm(range(X[0].shape[0]), desc='fed_learn_data_iter'):
              x_i = X[0][idx, :]
              # print('CHECKPOINT 0')
              encrypt_aggr_w, encrypt_aggr_b = clients[0].encrypted_gradient(idx, x_i, sum_to_w=None, sum_to_b=None)
              # print('CHECKPOINT 1')
              for i in range(1, config['n_clients']):
                  c = clients[i]
                  x_i = X[i][idx, :]
                  encrypt_aggr_w, encrypt_aggr_b = c.encrypted_gradient(idx, x_i, sum_to_w=encrypt_aggr_w, sum_to_b=encrypt_aggr_b)
              # print('CHECKPOINT 2')
              # Send aggregate to server and decrypt it
              aggr_w, aggr_b = server.decrypt_aggregate(encrypt_aggr_w, encrypt_aggr_b, n_clients)
              # print('CHECKPOINT 3')
              # Take gradient steps
              for c in clients:
                  c.gradient_step(aggr_w, aggr_b)

    print('Accuracy that each client gets after running the protocol:')
    start = time.time()
    for c in clients:
        y_pred = c.predict(X_test)
        hinge_l = accuracy_score(y_test, y_pred)
        print('{:s}:\t{:.2f}'.format(c.name, hinge_l))
    end = time.time()
    print("Time required for Single Prediction (Federated learning): ", (end-start)/(X_test.shape[0]))


def local_learning(X, y, X_test, y_test, config):
    n_clients = config['n_clients']
    names = ['Hospital {}'.format(i) for i in range(1, n_clients + 1)]

    # Instantiate the clients.
    # Each client gets the public key at creation and its own local dataset
    clients = []
    for i in tqdm(range(n_clients), desc='local_learn_client_number'):
        clients.append(Client(names[i], X[i], y[i], None))

    # Each client trains a linear regressor on its own data
    print('Accuracy that each client gets on test set by '
          'training only on own local data:')
    start = time.time()
    for c in clients:
        c.fit(config['n_iter'])
        y_pred = c.predict(X_test)
        hinge_l = accuracy_score(y_test, y_pred)
        print('{:s}:\t{:.2f}'.format(c.name, hinge_l))
    end = time.time()
    print("Time required for Single Prediction (Local learning): ", (end-start)/(X_test.shape[0]))


if __name__ == '__main__':
    config = {
        'n_clients': 5,
        'key_length': 128,
        'n_iter': 50,
        'eta': 1.5,
    }
    # load data, train/test split and split training data between clients
    download_data()
    X, y, X_test, y_test = preprocess_data(config)
    print(X[0].shape, y[0].shape, X_test.shape, y_test.shape)
    # first each hospital learns a model on its respective dataset for comparison.
    print("######################## LOCAL LEARNING (SVM) ###############################")
    local_learning(X, y, X_test, y_test, config)
    # and now the full glory of federated learning
    print("######################## FEDERATED LEARNING (SVM) ###############################")
    start = time.time()
    federated_learning(X, y, X_test, y_test, config)
    end = time.time()
    print("Time required for Federated Learning Process: ", (end-start)/(X_test.shape[0]))

Importing dataset from disk...
Vocabulary size: 7997
(50, 7997) (10979, 7997)
Labels in trainset are 0.22 spam : 0.78 ham
(10, 7997) (10,) (10979, 7997) (10979,)
######################## LOCAL LEARNING (SVM) ###############################


local_learn_client_number: 100%|██████████| 5/5 [00:00<00:00, 22453.45it/s]


Accuracy that each client gets on test set by training only on own local data:


local_learn_iters: 100%|██████████| 50/50 [00:00<00:00, 2724.00it/s]


Hospital 1:	0.71


...

Hospital 4:	0.85


local_learn_iters: 100%|██████████| 50/50 [00:00<00:00, 2118.31it/s]


Hospital 5:	0.74
Time required for Single Prediction (Local learning):  0.00014709266702902317
######################## FEDERATED LEARNING (SVM) ###############################
Running distributed gradient aggregation for 50 iterations


fed_learn_iters: 100%|██████████| 50/50 [44:57<00:00, 53.94s/it]


Accuracy that each client gets after running the protocol:
Hospital 1:	0.83
Hospital 2:	0.83
Hospital 3:	0.83
Hospital 4:	0.83
Hospital 5:	0.83
Time required for Single Prediction (Federated learning):  0.00010633208037443375
Time required for Federated Learning Process:  0.24577568445779857
